# Advanced Trading Strategies with HMM Regime Detection

This notebook demonstrates sophisticated trading strategies and portfolio management techniques using Hidden Markov Models for market regime detection.

## Learning Objectives
By the end of this tutorial, you will understand:
1. Multi-asset regime correlation analysis
2. Dynamic position sizing and risk management
3. Regime-based portfolio optimization
4. Advanced trading signals and strategy combinations
5. Performance attribution and risk decomposition
6. Real-world implementation considerations

## Prerequisites
- Completion of previous HMM tutorials (recommended)
- Understanding of portfolio theory basics
- Familiarity with risk management concepts

Let's begin by importing libraries and setting up our environment.

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta
from scipy import optimize
import warnings
warnings.filterwarnings('ignore')

# Import our HMM models
import sys
sys.path.append('../../')
from hidden_regime.models.base_hmm import HiddenMarkovModel
from hidden_regime.models.online_hmm import OnlineHMM, OnlineHMMConfig
from hidden_regime.data.loader import DataLoader

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)

# Portfolio analysis libraries
try:
    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA
    SKLEARN_AVAILABLE = True
except ImportError:
    print("⚠️ scikit-learn not available - some advanced features will be limited")
    SKLEARN_AVAILABLE = False

print("✅ Libraries imported successfully!")
print(f"Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Advanced features available: {SKLEARN_AVAILABLE}")

## 1. Multi-Asset Data Collection and Preprocessing

For advanced trading strategies, we need to analyze multiple assets simultaneously to understand regime correlations and build diversified portfolios.

In [ ]:
# Define our universe of assets
ASSET_UNIVERSE = {
    # Equity Indices
    'SPY': 'S&P 500 ETF',
    'QQQ': 'NASDAQ-100 ETF', 
    'IWM': 'Russell 2000 ETF',
    'VTI': 'Total Stock Market ETF',
    
    # International Equity
    'VEA': 'Developed Markets ETF',
    'VWO': 'Emerging Markets ETF',
    
    # Fixed Income
    'TLT': '20+ Year Treasury ETF',
    'SHY': '1-3 Year Treasury ETF',
    'LQD': 'Investment Grade Corp Bond ETF',
    
    # Commodities
    'GLD': 'Gold ETF',
    'USO': 'Oil ETF',
    
    # Volatility
    'VIX': 'VIX Volatility Index'
}

print(f"📊 MULTI-ASSET DATA COLLECTION")
print(f"=" * 40)
print(f"Asset universe: {len(ASSET_UNIVERSE)} instruments")

# Download data for all assets
start_date = "2022-01-01"
end_date = "2024-01-01"
tickers = list(ASSET_UNIVERSE.keys())

print(f"\nDownloading data from {start_date} to {end_date}...")

# Download data with error handling
asset_data = {}
failed_downloads = []

for ticker in tickers:
    try:
        print(f"  Downloading {ticker} ({ASSET_UNIVERSE[ticker]})...")
        data = yf.download(ticker, start=start_date, end=end_date, progress=False)
        
        if len(data) > 100:  # Minimum data requirement
            asset_data[ticker] = data
            print(f"    ✅ {len(data)} observations")
        else:
            print(f"    ❌ Insufficient data ({len(data)} observations)")
            failed_downloads.append(ticker)
    
    except Exception as e:
        print(f"    ❌ Download failed: {e}")
        failed_downloads.append(ticker)

# Remove failed downloads from universe
for ticker in failed_downloads:
    del ASSET_UNIVERSE[ticker]

print(f"\n✅ Successfully downloaded {len(asset_data)} assets")
if failed_downloads:
    print(f"⚠️ Failed downloads: {', '.join(failed_downloads)}")

# Create unified dataset
print(f"\n🔄 Creating unified dataset...")

# Extract adjusted close prices and calculate returns
price_data = pd.DataFrame()
return_data = pd.DataFrame()

for ticker, data in asset_data.items():
    price_data[ticker] = data['Adj Close']
    return_data[ticker] = np.log(data['Adj Close'] / data['Adj Close'].shift(1))

# Remove any remaining NaN values
price_data = price_data.dropna()
return_data = return_data.dropna()

print(f"✅ Unified dataset created:")
print(f"  Date range: {price_data.index[0].date()} to {price_data.index[-1].date()}")
print(f"  Total observations: {len(price_data)}")
print(f"  Assets: {len(price_data.columns)}")

# Display basic statistics
print(f"\n📈 RETURN STATISTICS (Annualized):")
stats_summary = pd.DataFrame({
    'Mean': return_data.mean() * 252,
    'Volatility': return_data.std() * np.sqrt(252),
    'Sharpe': (return_data.mean() / return_data.std()) * np.sqrt(252)
})

# Display top performers by Sharpe ratio
print(stats_summary.round(3).sort_values('Sharpe', ascending=False))

## 2. Multi-Asset Regime Detection

Let's detect regimes for multiple assets simultaneously to understand cross-asset regime behavior and correlations.

In [ ]:
# Multi-asset regime detection
print(f"🔍 MULTI-ASSET REGIME DETECTION")
print(f"=" * 40)

# Select core assets for regime detection (avoid overfitting)
core_assets = ['SPY', 'QQQ', 'TLT', 'GLD']  # Equity, Tech, Bonds, Gold
available_core_assets = [asset for asset in core_assets if asset in return_data.columns]

if len(available_core_assets) < 2:
    # Fallback to available assets
    available_core_assets = list(return_data.columns)[:4]

print(f"Core assets for regime detection: {available_core_assets}")

# Train individual HMMs for each asset
asset_hmms = {}
asset_regimes = pd.DataFrame()
asset_regime_probs = {}

print(f"\n🔄 Training individual HMMs...")

for asset in available_core_assets:
    print(f"  Training HMM for {asset}...")
    
    try:
        # Prepare data
        asset_returns = return_data[asset].dropna().values.reshape(-1, 1)
        
        # Train HMM
        hmm = HiddenMarkovModel(n_states=3, random_state=42)
        hmm.fit(asset_returns)
        
        # Get regime classifications
        states = hmm.predict(asset_returns)
        probs = hmm.predict_proba(asset_returns)
        
        # Store results
        asset_hmms[asset] = hmm
        asset_regimes[f'{asset}_regime'] = pd.Series(states, index=return_data[asset].dropna().index)
        asset_regime_probs[asset] = pd.DataFrame(probs, 
                                               index=return_data[asset].dropna().index,
                                               columns=[f'{asset}_prob_{i}' for i in range(3)])
        
        # Analyze regime characteristics
        means = hmm.emission_means_.flatten()
        stds = np.sqrt(hmm.emission_covariances_[:, 0, 0])
        
        print(f"    ✅ Trained successfully")
        print(f"       Regime 0: μ={means[0]:.4f}, σ={stds[0]:.4f}")
        print(f"       Regime 1: μ={means[1]:.4f}, σ={stds[1]:.4f}")
        print(f"       Regime 2: μ={means[2]:.4f}, σ={stds[2]:.4f}")
        
    except Exception as e:
        print(f"    ❌ Training failed: {e}")
        continue

print(f"\n✅ Trained HMMs for {len(asset_hmms)} assets")

# Analyze regime correlations
if len(asset_regimes.columns) >= 2:
    print(f"\n📊 REGIME CORRELATION ANALYSIS:")
    
    # Calculate regime correlation matrix
    regime_corr = asset_regimes.corr()
    
    print(f"\nRegime correlation matrix:")
    print(regime_corr.round(3))
    
    # Identify highly correlated regime pairs
    high_corr_pairs = []
    for i in range(len(regime_corr.columns)):
        for j in range(i+1, len(regime_corr.columns)):
            corr = regime_corr.iloc[i, j]
            if abs(corr) > 0.3:
                high_corr_pairs.append((regime_corr.columns[i], regime_corr.columns[j], corr))
    
    print(f"\nHigh regime correlations (|corr| > 0.3):")
    for asset1, asset2, corr in high_corr_pairs:
        print(f"  {asset1.replace('_regime', '')} ↔ {asset2.replace('_regime', '')}: {corr:.3f}")

# Create regime summary
print(f"\n📋 REGIME DISTRIBUTION SUMMARY:")
regime_summary = pd.DataFrame()

for col in asset_regimes.columns:
    asset_name = col.replace('_regime', '')
    regime_dist = asset_regimes[col].value_counts().sort_index()
    regime_pct = regime_dist / len(asset_regimes[col]) * 100
    
    print(f"\n{asset_name}:")
    for regime, count in regime_dist.items():
        pct = regime_pct[regime]
        print(f"  State {regime}: {count} days ({pct:.1f}%)")
    
    regime_summary[asset_name] = regime_pct

print(f"\n✅ Multi-asset regime detection completed!")

In [ ]:
# Visualize multi-asset regime analysis
if len(asset_regimes.columns) >= 2:
    fig, axes = plt.subplots(3, 1, figsize=(16, 14))
    fig.suptitle('Multi-Asset Regime Analysis', fontsize=16, fontweight='bold')
    
    # Plot 1: Regime correlation heatmap
    if len(regime_corr) > 1:
        # Clean up column names for display
        display_corr = regime_corr.copy()
        display_corr.columns = [col.replace('_regime', '') for col in display_corr.columns]
        display_corr.index = [idx.replace('_regime', '') for idx in display_corr.index]
        
        sns.heatmap(display_corr, annot=True, cmap='RdBu_r', center=0, 
                   square=True, ax=axes[0], cbar_kws={'label': 'Correlation'})
        axes[0].set_title('Cross-Asset Regime Correlations')
    
    # Plot 2: Regime distribution comparison
    if not regime_summary.empty:
        regime_summary.T.plot(kind='bar', ax=axes[1], 
                             color=['red', 'orange', 'green'])
        axes[1].set_title('Regime Distribution by Asset (%)')
        axes[1].set_ylabel('Percentage of Time')
        axes[1].legend(['State 0', 'State 1', 'State 2'])
        axes[1].tick_params(axis='x', rotation=45)
    
    # Plot 3: Regime synchronization over time
    if len(available_core_assets) >= 2:
        # Calculate regime agreement (simplified)
        primary_asset = available_core_assets[0]
        secondary_asset = available_core_assets[1]
        
        primary_regimes = asset_regimes[f'{primary_asset}_regime']
        secondary_regimes = asset_regimes[f'{secondary_asset}_regime']
        
        # Find common dates
        common_dates = primary_regimes.index.intersection(secondary_regimes.index)
        if len(common_dates) > 0:
            primary_common = primary_regimes.loc[common_dates]
            secondary_common = secondary_regimes.loc[common_dates]
            
            # Calculate rolling regime agreement
            agreement = (primary_common == secondary_common).astype(int)
            rolling_agreement = agreement.rolling(window=30, min_periods=10).mean()
            
            axes[2].plot(rolling_agreement.index, rolling_agreement.values, 
                        linewidth=2, color='purple', alpha=0.8)
            axes[2].axhline(y=0.5, color='gray', linestyle='--', alpha=0.7, 
                          label='Random Agreement (50%)')
            axes[2].set_title(f'Regime Agreement: {primary_asset} vs {secondary_asset} (30-day rolling)')
            axes[2].set_ylabel('Agreement Rate')
            axes[2].set_ylim(0, 1)
            axes[2].legend()
            axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Additional analysis: Regime transition analysis
    print(f"\n🔄 REGIME TRANSITION ANALYSIS:")
    print(f"=" * 30)
    
    for asset in available_core_assets[:2]:  # Analyze first 2 assets
        if f'{asset}_regime' in asset_regimes.columns:
            regimes = asset_regimes[f'{asset}_regime']
            transitions = (regimes != regimes.shift(1)).sum() - 1  # -1 for first observation
            avg_duration = len(regimes) / (transitions + 1) if transitions > 0 else len(regimes)
            
            print(f"\n{asset}:")
            print(f"  Total regime transitions: {transitions}")
            print(f"  Average regime duration: {avg_duration:.1f} days")
            
            # Regime persistence analysis
            if asset in asset_hmms:
                transition_matrix = asset_hmms[asset].transition_matrix_
                persistence_probs = np.diag(transition_matrix)
                
                print(f"  Regime persistence probabilities:")
                for i, prob in enumerate(persistence_probs):
                    expected_duration = 1 / (1 - prob) if prob < 1 else float('inf')
                    print(f"    State {i}: {prob:.3f} (expected duration: {expected_duration:.1f} days)")

else:
    print("❌ Insufficient regime data for visualization")

## 3. Dynamic Position Sizing and Risk Management

Let's implement sophisticated position sizing algorithms that adapt based on regime detection and risk levels.

In [ ]:
# Advanced position sizing and risk management
print(f"⚖️ DYNAMIC POSITION SIZING & RISK MANAGEMENT")
print(f"=" * 50)

class AdvancedPositionSizer:
    """
    Advanced position sizing with regime-based risk management
    """
    
    def __init__(self, base_capital=1000000):
        self.base_capital = base_capital
        
        # Risk parameters by regime
        self.regime_risk_params = {
            0: {'max_position': 0.3, 'volatility_target': 0.15, 'drawdown_limit': 0.05},  # Conservative
            1: {'max_position': 0.6, 'volatility_target': 0.20, 'drawdown_limit': 0.08},  # Moderate  
            2: {'max_position': 1.0, 'volatility_target': 0.25, 'drawdown_limit': 0.12},  # Aggressive
        }
        
        # Kelly criterion parameters
        self.kelly_lookback = 252  # 1 year for Kelly estimation
        self.kelly_max_position = 0.25  # Maximum Kelly position
        
        # Risk monitoring
        self.var_confidence = 0.05  # 5% VaR
        self.var_lookback = 252
        
    def calculate_regime_based_position(self, asset, regime, confidence, 
                                      historical_returns, current_volatility):
        """
        Calculate position size based on regime and risk parameters
        """
        # Get risk parameters for current regime
        risk_params = self.regime_risk_params.get(regime, self.regime_risk_params[1])
        
        # Base position from regime
        max_position = risk_params['max_position']
        volatility_target = risk_params['volatility_target']
        
        # Volatility scaling
        if current_volatility > 0:
            vol_scaling = min(1.5, volatility_target / current_volatility)
        else:
            vol_scaling = 1.0
        
        # Confidence scaling
        confidence_scaling = max(0.2, confidence)  # Minimum 20% when uncertain
        
        # Calculate base position
        base_position = max_position * vol_scaling * confidence_scaling
        
        return min(max_position, base_position)
    
    def calculate_kelly_position(self, historical_returns, lookback=None):
        """
        Calculate Kelly criterion optimal position size
        """
        if lookback is None:
            lookback = self.kelly_lookback
        
        if len(historical_returns) < lookback:
            returns = historical_returns
        else:
            returns = historical_returns[-lookback:]
        
        if len(returns) < 30:  # Minimum sample size
            return 0.1  # Conservative fallback
        
        mean_return = np.mean(returns)
        variance = np.var(returns)
        
        if variance <= 0:
            return 0.1
        
        # Kelly fraction: f = (mean - risk_free_rate) / variance
        # Assuming risk-free rate ≈ 0 for simplicity
        kelly_fraction = mean_return / variance
        
        # Apply Kelly fraction bounds
        kelly_position = max(0, min(self.kelly_max_position, kelly_fraction))
        
        return kelly_position
    
    def calculate_var_position(self, historical_returns, confidence=None):
        """
        Calculate position size based on Value at Risk constraint
        """
        if confidence is None:
            confidence = self.var_confidence
        
        if len(historical_returns) < 30:
            return 0.1  # Conservative fallback
        
        # Calculate VaR
        var = np.percentile(historical_returns, confidence * 100)
        
        # Target maximum loss of 2% of capital
        target_max_loss = 0.02
        
        if var >= 0:  # No downside risk observed
            return 1.0
        
        # Position size to limit VaR to target
        var_position = target_max_loss / abs(var)
        
        return min(1.0, var_position)
    
    def calculate_optimal_position(self, asset, regime, confidence, 
                                 historical_returns, current_volatility=None):
        """
        Calculate optimal position combining multiple approaches
        """
        if len(historical_returns) == 0:
            return 0.0
        
        # Calculate current volatility if not provided
        if current_volatility is None:
            current_volatility = np.std(historical_returns[-30:]) * np.sqrt(252) if len(historical_returns) >= 30 else 0.2
        
        # Calculate position sizes using different methods
        regime_position = self.calculate_regime_based_position(
            asset, regime, confidence, historical_returns, current_volatility)
        
        kelly_position = self.calculate_kelly_position(historical_returns)
        
        var_position = self.calculate_var_position(historical_returns)
        
        # Combine positions (conservative approach - take minimum)
        positions = [regime_position, kelly_position, var_position]
        optimal_position = min(positions)
        
        return {
            'optimal_position': optimal_position,
            'regime_position': regime_position,
            'kelly_position': kelly_position,
            'var_position': var_position,
            'current_volatility': current_volatility
        }

# Initialize position sizer
position_sizer = AdvancedPositionSizer(base_capital=1000000)

print(f"✅ Advanced Position Sizer initialized")
print(f"  Base capital: ${position_sizer.base_capital:,}")
print(f"  Regime risk parameters configured")
print(f"  Kelly lookback: {position_sizer.kelly_lookback} days")
print(f"  VaR confidence: {position_sizer.var_confidence:.1%}")

# Demonstrate position sizing for available assets
print(f"\n📊 POSITION SIZING EXAMPLES:")
print(f"=" * 30)

for asset in available_core_assets[:2]:  # Demo on first 2 assets
    if asset in asset_regimes.columns or f'{asset}_regime' in asset_regimes.columns:
        # Get latest regime information
        regime_col = f'{asset}_regime' if f'{asset}_regime' in asset_regimes.columns else None
        
        if regime_col and len(asset_regimes[regime_col]) > 0:
            latest_regime = asset_regimes[regime_col].iloc[-1]
            
            # Get historical returns
            historical_returns = return_data[asset].dropna().values
            
            # Calculate positions for different confidence levels
            confidence_levels = [0.5, 0.7, 0.9]
            
            print(f"\n{asset} (Current Regime: {latest_regime}):")
            
            for confidence in confidence_levels:
                position_info = position_sizer.calculate_optimal_position(
                    asset, latest_regime, confidence, historical_returns
                )
                
                print(f"  Confidence {confidence:.0%}:")
                print(f"    Optimal position: {position_info['optimal_position']:.1%}")
                print(f"    Regime-based: {position_info['regime_position']:.1%}")
                print(f"    Kelly criterion: {position_info['kelly_position']:.1%}")
                print(f"    VaR-based: {position_info['var_position']:.1%}")
                print(f"    Current volatility: {position_info['current_volatility']:.1%}")

print(f"\n✅ Position sizing analysis completed!")

## 4. Regime-Based Portfolio Optimization

Now let's build a portfolio optimization framework that adapts allocation based on regime detection across multiple assets.

In [ ]:
# Advanced portfolio optimization with regime awareness
print(f"📈 REGIME-BASED PORTFOLIO OPTIMIZATION")
print(f"=" * 45)

class RegimeAwarePortfolioOptimizer:
    """
    Portfolio optimizer that adapts based on regime detection
    """
    
    def __init__(self, assets, return_data, regime_data=None):
        self.assets = assets
        self.return_data = return_data[assets].copy()
        self.regime_data = regime_data
        
        # Portfolio constraints
        self.min_weight = 0.0   # Minimum weight per asset
        self.max_weight = 0.5   # Maximum weight per asset  
        self.max_total_weight = 1.0  # Total portfolio weight
        
        # Risk parameters
        self.risk_aversion = 1.0  # Risk aversion parameter
        self.regime_adjustment = True
        
    def calculate_regime_adjusted_returns(self, lookback_days=252):
        """
        Calculate expected returns adjusted for current regime
        """
        expected_returns = pd.Series(index=self.assets, dtype=float)
        
        for asset in self.assets:
            asset_returns = self.return_data[asset].dropna()
            
            if len(asset_returns) == 0:
                expected_returns[asset] = 0.0
                continue
            
            # Use recent returns as baseline
            recent_returns = asset_returns.tail(min(lookback_days, len(asset_returns)))
            base_return = recent_returns.mean()
            
            # Adjust based on regime if available
            if (self.regime_data is not None and 
                f'{asset}_regime' in self.regime_data.columns and
                len(self.regime_data[f'{asset}_regime']) > 0):
                
                current_regime = self.regime_data[f'{asset}_regime'].iloc[-1]
                
                # Regime-based return adjustments
                regime_adjustments = {0: -0.5, 1: 0.0, 2: 0.5}  # Bear, Neutral, Bull
                adjustment = regime_adjustments.get(current_regime, 0.0)
                
                expected_returns[asset] = base_return * (1 + adjustment)
            else:
                expected_returns[asset] = base_return
        
        return expected_returns
    
    def calculate_regime_adjusted_covariance(self, lookback_days=252):
        """
        Calculate covariance matrix with regime adjustments
        """
        # Get recent returns
        recent_data = self.return_data.tail(min(lookback_days, len(self.return_data)))
        base_cov = recent_data.cov() * 252  # Annualized
        
        if not self.regime_adjustment or self.regime_data is None:
            return base_cov
        
        # Adjust covariance based on regime uncertainty
        adjusted_cov = base_cov.copy()
        
        for asset in self.assets:
            if f'{asset}_regime' in self.regime_data.columns:
                # Get regime probabilities if available (simplified)
                # In practice, you would use actual regime probabilities
                regime_uncertainty = 0.1  # Placeholder for regime uncertainty
                
                # Increase variance during regime uncertainty
                adjusted_cov.loc[asset, asset] *= (1 + regime_uncertainty)
        
        return adjusted_cov
    
    def optimize_portfolio(self, method='mean_variance', target_return=None):
        """
        Optimize portfolio weights based on specified method
        """
        # Calculate expected returns and covariance
        expected_returns = self.calculate_regime_adjusted_returns()
        cov_matrix = self.calculate_regime_adjusted_covariance()
        
        n_assets = len(self.assets)
        
        if method == 'equal_weight':
            weights = np.ones(n_assets) / n_assets
            
        elif method == 'mean_variance':
            weights = self._optimize_mean_variance(expected_returns, cov_matrix, target_return)
            
        elif method == 'min_variance':
            weights = self._optimize_min_variance(cov_matrix)
            
        elif method == 'max_sharpe':
            weights = self._optimize_max_sharpe(expected_returns, cov_matrix)
            
        elif method == 'risk_parity':
            weights = self._optimize_risk_parity(cov_matrix)
            
        else:
            raise ValueError(f"Unknown optimization method: {method}")
        
        # Create results dictionary
        portfolio_weights = pd.Series(weights, index=self.assets)
        
        # Calculate portfolio metrics
        portfolio_return = np.sum(portfolio_weights * expected_returns) * 252
        portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
        portfolio_sharpe = portfolio_return / portfolio_risk if portfolio_risk > 0 else 0
        
        return {
            'weights': portfolio_weights,
            'expected_return': portfolio_return,
            'risk': portfolio_risk,
            'sharpe_ratio': portfolio_sharpe,
            'method': method
        }
    
    def _optimize_mean_variance(self, expected_returns, cov_matrix, target_return=None):
        """
        Mean-variance optimization
        """
        n_assets = len(self.assets)
        
        # Objective: minimize portfolio variance
        def objective(weights):
            return np.dot(weights, np.dot(cov_matrix, weights))
        
        # Constraints
        constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]  # Weights sum to 1
        
        if target_return is not None:
            constraints.append({
                'type': 'eq', 
                'fun': lambda w: np.sum(w * expected_returns) - target_return/252
            })
        
        # Bounds for weights
        bounds = tuple((self.min_weight, self.max_weight) for _ in range(n_assets))
        
        # Initial guess
        x0 = np.ones(n_assets) / n_assets
        
        # Optimize
        try:
            result = optimize.minimize(objective, x0, method='SLSQP', 
                                     bounds=bounds, constraints=constraints)
            
            if result.success:
                return result.x
            else:
                print(f"⚠️ Mean-variance optimization failed: {result.message}")
                return np.ones(n_assets) / n_assets
        except:
            print(f"⚠️ Mean-variance optimization error, using equal weights")
            return np.ones(n_assets) / n_assets
    
    def _optimize_min_variance(self, cov_matrix):
        """
        Minimum variance optimization
        """
        n_assets = len(self.assets)
        
        # Objective: minimize portfolio variance
        def objective(weights):
            return np.dot(weights, np.dot(cov_matrix, weights))
        
        # Constraints
        constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
        
        # Bounds
        bounds = tuple((self.min_weight, self.max_weight) for _ in range(n_assets))
        
        # Initial guess
        x0 = np.ones(n_assets) / n_assets
        
        try:
            result = optimize.minimize(objective, x0, method='SLSQP', 
                                     bounds=bounds, constraints=constraints)
            return result.x if result.success else np.ones(n_assets) / n_assets
        except:
            return np.ones(n_assets) / n_assets
    
    def _optimize_max_sharpe(self, expected_returns, cov_matrix):
        """
        Maximum Sharpe ratio optimization
        """
        n_assets = len(self.assets)
        
        # Objective: minimize negative Sharpe ratio
        def objective(weights):
            portfolio_return = np.sum(weights * expected_returns)
            portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
            return -portfolio_return / portfolio_risk if portfolio_risk > 0 else -1e10
        
        # Constraints and bounds
        constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
        bounds = tuple((self.min_weight, self.max_weight) for _ in range(n_assets))
        
        # Initial guess
        x0 = np.ones(n_assets) / n_assets
        
        try:
            result = optimize.minimize(objective, x0, method='SLSQP', 
                                     bounds=bounds, constraints=constraints)
            return result.x if result.success else np.ones(n_assets) / n_assets
        except:
            return np.ones(n_assets) / n_assets
    
    def _optimize_risk_parity(self, cov_matrix):
        """
        Risk parity optimization (simplified)
        """
        n_assets = len(self.assets)
        
        # Start with inverse volatility weighting as approximation
        volatilities = np.sqrt(np.diag(cov_matrix))
        inv_vol_weights = 1 / volatilities
        weights = inv_vol_weights / np.sum(inv_vol_weights)
        
        # Apply constraints
        weights = np.clip(weights, self.min_weight, self.max_weight)
        weights = weights / np.sum(weights)  # Renormalize
        
        return weights

# Initialize portfolio optimizer
optimizer_assets = [asset for asset in available_core_assets if asset in return_data.columns]
if len(optimizer_assets) >= 2:
    portfolio_optimizer = RegimeAwarePortfolioOptimizer(
        assets=optimizer_assets,
        return_data=return_data,
        regime_data=asset_regimes
    )
    
    print(f"✅ Portfolio optimizer initialized")
    print(f"  Assets: {optimizer_assets}")
    print(f"  Data period: {return_data.index[0].date()} to {return_data.index[-1].date()}")
    print(f"  Regime adjustment: {portfolio_optimizer.regime_adjustment}")
    
    # Test different optimization methods
    optimization_methods = ['equal_weight', 'min_variance', 'max_sharpe', 'risk_parity']
    
    print(f"\n📊 PORTFOLIO OPTIMIZATION RESULTS:")
    print(f"=" * 40)
    
    optimization_results = {}
    
    for method in optimization_methods:
        print(f"\n🔄 Optimizing using {method.replace('_', ' ').title()} method...")
        
        try:
            result = portfolio_optimizer.optimize_portfolio(method=method)
            optimization_results[method] = result
            
            print(f"  Expected Return: {result['expected_return']:.2%}")
            print(f"  Risk (Volatility): {result['risk']:.2%}")
            print(f"  Sharpe Ratio: {result['sharpe_ratio']:.3f}")
            print(f"  Weights:")
            
            for asset, weight in result['weights'].items():
                print(f"    {asset}: {weight:.1%}")
        
        except Exception as e:
            print(f"  ❌ Optimization failed: {e}")
            continue
    
    print(f"\n✅ Portfolio optimization completed!")
    
else:
    print(f"❌ Insufficient assets for portfolio optimization")
    print(f"Available assets: {list(return_data.columns)}")
    optimization_results = {}

In [ ]:
# Visualize portfolio optimization results
if optimization_results:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Portfolio Optimization Results Comparison', fontsize=16, fontweight='bold')
    
    # Extract data for plotting
    methods = list(optimization_results.keys())
    returns = [optimization_results[method]['expected_return'] for method in methods]
    risks = [optimization_results[method]['risk'] for method in methods]
    sharpes = [optimization_results[method]['sharpe_ratio'] for method in methods]
    
    # Plot 1: Risk-Return scatter
    colors = ['red', 'blue', 'green', 'orange']
    for i, method in enumerate(methods):
        axes[0, 0].scatter(risks[i], returns[i], s=100, alpha=0.7, 
                          color=colors[i % len(colors)], label=method.replace('_', ' ').title())
    
    axes[0, 0].set_xlabel('Risk (Volatility)')
    axes[0, 0].set_ylabel('Expected Return')
    axes[0, 0].set_title('Risk-Return Profile')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot 2: Sharpe ratios comparison
    bars = axes[0, 1].bar(range(len(methods)), sharpes, 
                         color=colors[:len(methods)], alpha=0.7)
    axes[0, 1].set_xlabel('Optimization Method')
    axes[0, 1].set_ylabel('Sharpe Ratio')
    axes[0, 1].set_title('Sharpe Ratio Comparison')
    axes[0, 1].set_xticks(range(len(methods)))
    axes[0, 1].set_xticklabels([m.replace('_', ' ').title() for m in methods], rotation=45)
    axes[0, 1].grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar, sharpe in zip(bars, sharpes):
        height = bar.get_height()
        axes[0, 1].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                       f'{sharpe:.2f}', ha='center', va='bottom')
    
    # Plot 3: Weight allocation for best Sharpe method
    best_sharpe_method = methods[np.argmax(sharpes)]
    best_weights = optimization_results[best_sharpe_method]['weights']
    
    wedges, texts, autotexts = axes[1, 0].pie(best_weights.values, 
                                             labels=best_weights.index,
                                             autopct='%1.1f%%',
                                             startangle=90)
    axes[1, 0].set_title(f'Asset Allocation: {best_sharpe_method.replace("_", " ").title()}')
    
    # Plot 4: Weight comparison across methods
    if len(optimizer_assets) <= 4:  # Only plot if manageable number of assets
        x = np.arange(len(optimizer_assets))
        width = 0.2
        
        for i, method in enumerate(methods):
            weights = [optimization_results[method]['weights'][asset] for asset in optimizer_assets]
            axes[1, 1].bar(x + i*width, weights, width, 
                          label=method.replace('_', ' ').title(), 
                          color=colors[i % len(colors)], alpha=0.7)
        
        axes[1, 1].set_xlabel('Assets')
        axes[1, 1].set_ylabel('Weight')
        axes[1, 1].set_title('Weight Comparison Across Methods')
        axes[1, 1].set_xticks(x + width * (len(methods) - 1) / 2)
        axes[1, 1].set_xticklabels(optimizer_assets)
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
    else:
        axes[1, 1].text(0.5, 0.5, 'Too many assets\nto display clearly', 
                       ha='center', va='center', transform=axes[1, 1].transAxes,
                       fontsize=12)
        axes[1, 1].set_title('Weight Comparison (Too Many Assets)')
    
    plt.tight_layout()
    plt.show()
    
    # Summary table
    print(f"\n📊 OPTIMIZATION SUMMARY TABLE:")
    print(f"=" * 60)
    
    summary_df = pd.DataFrame({
        'Method': [m.replace('_', ' ').title() for m in methods],
        'Expected Return': [f"{r:.2%}" for r in returns],
        'Risk': [f"{r:.2%}" for r in risks],
        'Sharpe Ratio': [f"{s:.3f}" for s in sharpes]
    })
    
    print(summary_df.to_string(index=False))
    
    # Best method recommendation
    best_method = methods[np.argmax(sharpes)]
    print(f"\n🏆 Best Method by Sharpe Ratio: {best_method.replace('_', ' ').title()}")
    print(f"   Sharpe Ratio: {max(sharpes):.3f}")
    print(f"   Expected Return: {returns[np.argmax(sharpes)]:.2%}")
    print(f"   Risk: {risks[np.argmax(sharpes)]:.2%}")
    
else:
    print("❌ No optimization results to visualize")

## 5. Advanced Trading Strategy Implementation

Let's implement a comprehensive trading system that combines regime detection, dynamic position sizing, and portfolio optimization.

In [ ]:
# Comprehensive advanced trading strategy
print(f"🚀 ADVANCED TRADING STRATEGY IMPLEMENTATION")
print(f"=" * 50)

class AdvancedTradingStrategy:
    """
    Comprehensive trading strategy combining multiple techniques
    """
    
    def __init__(self, assets, initial_capital=1000000):
        self.assets = assets
        self.initial_capital = initial_capital
        self.current_capital = initial_capital
        
        # Strategy components
        self.position_sizer = AdvancedPositionSizer(initial_capital)
        self.portfolio_optimizer = None
        
        # Current positions and state
        self.current_positions = pd.Series(0.0, index=assets)
        self.regime_history = pd.DataFrame()
        self.position_history = pd.DataFrame()
        self.performance_history = []
        
        # Strategy parameters
        self.rebalance_frequency = 5  # Rebalance every N days
        self.transaction_cost = 0.001  # 10 bps transaction cost
        self.minimum_trade_size = 0.01  # Minimum 1% position change
        
        # Risk management
        self.max_portfolio_leverage = 1.0
        self.stop_loss_threshold = -0.02  # 2% stop loss
        self.take_profit_threshold = 0.05  # 5% take profit
        
    def initialize_hmms(self, return_data, lookback_days=252):
        """
        Initialize HMMs for all assets
        """
        self.asset_hmms = {}
        
        print(f"🔄 Initializing HMMs for {len(self.assets)} assets...")
        
        for asset in self.assets:
            if asset in return_data.columns:
                try:
                    asset_returns = return_data[asset].dropna().tail(lookback_days)
                    
                    if len(asset_returns) > 50:  # Minimum data requirement
                        hmm = HiddenMarkovModel(n_states=3, random_state=42)
                        hmm.fit(asset_returns.values.reshape(-1, 1))
                        self.asset_hmms[asset] = hmm
                        print(f"  ✅ {asset}: HMM trained on {len(asset_returns)} observations")
                    else:
                        print(f"  ⚠️ {asset}: Insufficient data ({len(asset_returns)} observations)")
                
                except Exception as e:
                    print(f"  ❌ {asset}: HMM training failed - {e}")
        
        print(f"✅ Initialized {len(self.asset_hmms)} HMMs")
    
    def detect_regimes(self, return_data, current_date):
        """
        Detect current regimes for all assets
        """
        regime_info = {}
        
        for asset in self.assets:
            if asset in self.asset_hmms and asset in return_data.columns:
                try:
                    # Get recent returns for prediction
                    asset_data = return_data[asset].loc[:current_date].dropna()
                    if len(asset_data) > 0:
                        recent_returns = asset_data.tail(30).values.reshape(-1, 1)
                        
                        # Predict regime
                        states = self.asset_hmms[asset].predict(recent_returns)
                        probs = self.asset_hmms[asset].predict_proba(recent_returns)
                        
                        current_regime = states[-1]
                        current_confidence = np.max(probs[-1])
                        
                        regime_info[asset] = {
                            'regime': current_regime,
                            'confidence': current_confidence,
                            'probabilities': probs[-1]
                        }
                
                except Exception as e:
                    print(f"Warning: Regime detection failed for {asset}: {e}")
                    regime_info[asset] = {
                        'regime': 1,  # Default to neutral
                        'confidence': 0.5,
                        'probabilities': [0.33, 0.34, 0.33]
                    }
        
        return regime_info
    
    def calculate_target_positions(self, return_data, regime_info, current_date):
        """
        Calculate target positions based on regime and optimization
        """
        target_positions = pd.Series(0.0, index=self.assets)
        
        for asset in self.assets:
            if asset in regime_info and asset in return_data.columns:
                # Get historical returns
                historical_returns = return_data[asset].loc[:current_date].dropna().values
                
                if len(historical_returns) > 30:
                    # Calculate position using advanced position sizer
                    position_info = self.position_sizer.calculate_optimal_position(
                        asset=asset,
                        regime=regime_info[asset]['regime'],
                        confidence=regime_info[asset]['confidence'],
                        historical_returns=historical_returns
                    )
                    
                    # Determine direction based on regime
                    regime = regime_info[asset]['regime']
                    if regime == 0:  # Bear regime
                        direction = -1
                    elif regime == 1:  # Neutral regime
                        direction = 0
                    else:  # Bull regime
                        direction = 1
                    
                    target_positions[asset] = position_info['optimal_position'] * direction
        
        # Apply portfolio-level constraints
        total_abs_position = np.sum(np.abs(target_positions))
        if total_abs_position > self.max_portfolio_leverage:
            target_positions *= (self.max_portfolio_leverage / total_abs_position)
        
        return target_positions
    
    def execute_trades(self, target_positions, current_prices):
        """
        Execute trades to reach target positions
        """
        trades = []
        total_transaction_costs = 0
        
        for asset in self.assets:
            if asset in current_prices.index:
                target_position = target_positions[asset]
                current_position = self.current_positions[asset]
                position_change = target_position - current_position
                
                # Only trade if position change is significant
                if abs(position_change) > self.minimum_trade_size:
                    # Calculate transaction cost
                    trade_value = abs(position_change) * self.current_capital
                    transaction_cost = trade_value * self.transaction_cost
                    total_transaction_costs += transaction_cost
                    
                    trades.append({
                        'asset': asset,
                        'old_position': current_position,
                        'new_position': target_position,
                        'position_change': position_change,
                        'price': current_prices[asset],
                        'transaction_cost': transaction_cost
                    })
                    
                    # Update position
                    self.current_positions[asset] = target_position
        
        # Apply transaction costs to capital
        self.current_capital -= total_transaction_costs
        
        return trades, total_transaction_costs
    
    def calculate_portfolio_pnl(self, returns, date):
        """
        Calculate portfolio P&L for given returns
        """
        portfolio_return = 0
        
        for asset in self.assets:
            if asset in returns.index and asset in self.current_positions.index:
                asset_return = returns[asset]
                position = self.current_positions[asset]
                contribution = position * asset_return
                portfolio_return += contribution
        
        # Update capital
        pnl = self.current_capital * portfolio_return
        self.current_capital += pnl
        
        return portfolio_return, pnl
    
    def run_backtest(self, price_data, return_data, start_date=None, end_date=None):
        """
        Run comprehensive backtest
        """
        print(f"🔄 Running advanced trading strategy backtest...")
        
        # Set date range
        if start_date is None:
            start_date = return_data.index[100]  # Skip initial period for HMM training
        if end_date is None:
            end_date = return_data.index[-1]
        
        test_dates = return_data.loc[start_date:end_date].index
        print(f"Backtest period: {start_date.date()} to {end_date.date()} ({len(test_dates)} days)")
        
        # Initialize HMMs
        self.initialize_hmms(return_data.loc[:start_date])
        
        # Run backtest
        day_counter = 0
        
        for current_date in test_dates:
            try:
                # Detect regimes
                regime_info = self.detect_regimes(return_data, current_date)
                
                # Calculate target positions (rebalance periodically)
                if day_counter % self.rebalance_frequency == 0:
                    target_positions = self.calculate_target_positions(
                        return_data, regime_info, current_date)
                    
                    # Execute trades
                    current_prices = price_data.loc[current_date]
                    trades, transaction_costs = self.execute_trades(target_positions, current_prices)
                else:
                    trades = []
                    transaction_costs = 0
                
                # Calculate P&L
                daily_returns = return_data.loc[current_date]
                portfolio_return, pnl = self.calculate_portfolio_pnl(daily_returns, current_date)
                
                # Record performance
                performance_record = {
                    'date': current_date,
                    'capital': self.current_capital,
                    'portfolio_return': portfolio_return,
                    'pnl': pnl,
                    'transaction_costs': transaction_costs,
                    'num_trades': len(trades),
                    'total_position': np.sum(np.abs(self.current_positions))
                }
                
                # Add regime information
                for asset in self.assets:
                    if asset in regime_info:
                        performance_record[f'{asset}_regime'] = regime_info[asset]['regime']
                        performance_record[f'{asset}_confidence'] = regime_info[asset]['confidence']
                        performance_record[f'{asset}_position'] = self.current_positions[asset]
                
                self.performance_history.append(performance_record)
                
                day_counter += 1
                
                # Progress update
                if day_counter % 50 == 0:
                    total_return = (self.current_capital / self.initial_capital - 1) * 100
                    print(f"  Day {day_counter}: Capital = ${self.current_capital:,.0f} ({total_return:+.1f}%)")
            
            except Exception as e:
                print(f"  ❌ Error on {current_date}: {e}")
                continue
        
        print(f"✅ Backtest completed: {day_counter} days processed")
        
        # Create results DataFrame
        self.backtest_results = pd.DataFrame(self.performance_history)
        if not self.backtest_results.empty:
            self.backtest_results.set_index('date', inplace=True)
        
        return self.backtest_results
    
    def calculate_performance_metrics(self):
        """
        Calculate comprehensive performance metrics
        """
        if self.backtest_results.empty:
            return {}
        
        results = self.backtest_results.copy()
        
        # Basic metrics
        total_return = (self.current_capital / self.initial_capital) - 1
        num_days = len(results)
        annualized_return = (1 + total_return) ** (252 / num_days) - 1
        
        # Risk metrics
        daily_returns = results['portfolio_return'].dropna()
        if len(daily_returns) > 0:
            volatility = daily_returns.std() * np.sqrt(252)
            sharpe_ratio = (annualized_return - 0.02) / volatility if volatility > 0 else 0
            
            # Drawdown analysis
            capital_series = results['capital']
            running_max = capital_series.expanding().max()
            drawdowns = (capital_series - running_max) / running_max
            max_drawdown = drawdowns.min()
            
            # Additional metrics
            positive_days = (daily_returns > 0).sum()
            win_rate = positive_days / len(daily_returns)
            
            avg_win = daily_returns[daily_returns > 0].mean() if positive_days > 0 else 0
            avg_loss = daily_returns[daily_returns < 0].mean() if (len(daily_returns) - positive_days) > 0 else 0
            profit_factor = abs(avg_win / avg_loss) if avg_loss != 0 else float('inf')
            
        else:
            volatility = sharpe_ratio = max_drawdown = win_rate = profit_factor = 0
            avg_win = avg_loss = 0
        
        # Trading metrics
        total_trades = results['num_trades'].sum()
        total_transaction_costs = results['transaction_costs'].sum()
        
        return {
            'total_return': total_return,
            'annualized_return': annualized_return,
            'volatility': volatility,
            'sharpe_ratio': sharpe_ratio,
            'max_drawdown': max_drawdown,
            'win_rate': win_rate,
            'profit_factor': profit_factor,
            'avg_win': avg_win,
            'avg_loss': avg_loss,
            'total_trades': total_trades,
            'total_transaction_costs': total_transaction_costs,
            'final_capital': self.current_capital,
            'trading_days': num_days
        }

# Initialize and run advanced trading strategy
if len(available_core_assets) >= 2:
    strategy = AdvancedTradingStrategy(
        assets=available_core_assets,
        initial_capital=1000000
    )
    
    print(f"✅ Advanced Trading Strategy initialized")
    print(f"  Assets: {strategy.assets}")
    print(f"  Initial capital: ${strategy.initial_capital:,}")
    print(f"  Rebalance frequency: {strategy.rebalance_frequency} days")
    print(f"  Transaction cost: {strategy.transaction_cost:.1%}")
    
else:
    print("❌ Insufficient assets for advanced trading strategy")
    strategy = None

In [ ]:
# Run the advanced trading strategy backtest
if strategy is not None:
    print(f"\n🚀 RUNNING ADVANCED STRATEGY BACKTEST")
    print(f"=" * 45)
    
    # Run backtest on available data
    try:
        backtest_results = strategy.run_backtest(
            price_data=price_data,
            return_data=return_data,
            start_date=return_data.index[150],  # Skip initial period
            end_date=return_data.index[-1]
        )
        
        if not backtest_results.empty:
            print(f"\n📊 BACKTEST COMPLETED SUCCESSFULLY")
            print(f"Observations: {len(backtest_results)}")
            print(f"Period: {backtest_results.index[0].date()} to {backtest_results.index[-1].date()}")
            
            # Calculate and display performance metrics
            performance_metrics = strategy.calculate_performance_metrics()
            
            print(f"\n📈 PERFORMANCE SUMMARY:")
            print(f"=" * 30)
            print(f"Initial Capital: ${strategy.initial_capital:,}")
            print(f"Final Capital: ${performance_metrics['final_capital']:,.0f}")
            print(f"Total Return: {performance_metrics['total_return']:.2%}")
            print(f"Annualized Return: {performance_metrics['annualized_return']:.2%}")
            print(f"Volatility: {performance_metrics['volatility']:.2%}")
            print(f"Sharpe Ratio: {performance_metrics['sharpe_ratio']:.3f}")
            print(f"Maximum Drawdown: {performance_metrics['max_drawdown']:.2%}")
            print(f"Win Rate: {performance_metrics['win_rate']:.1%}")
            print(f"Profit Factor: {performance_metrics['profit_factor']:.2f}")
            print(f"Total Trades: {performance_metrics['total_trades']:.0f}")
            print(f"Transaction Costs: ${performance_metrics['total_transaction_costs']:,.0f}")
            
            # Calculate buy-and-hold comparison
            # Use equal-weighted portfolio of assets as benchmark
            benchmark_weights = pd.Series(1/len(strategy.assets), index=strategy.assets)
            benchmark_returns = (return_data[strategy.assets].loc[backtest_results.index] * benchmark_weights).sum(axis=1)
            benchmark_total_return = (1 + benchmark_returns).prod() - 1
            
            print(f"\n⚖️ BENCHMARK COMPARISON:")
            print(f"Benchmark (Equal Weight): {benchmark_total_return:.2%}")
            print(f"Strategy Return: {performance_metrics['total_return']:.2%}")
            print(f"Excess Return: {performance_metrics['total_return'] - benchmark_total_return:.2%}")
            
            if performance_metrics['total_return'] > benchmark_total_return:
                print(f"✅ Strategy outperformed benchmark!")
            else:
                print(f"⚠️ Strategy underperformed benchmark")
            
        else:
            print(f"❌ Backtest failed - no results generated")
            
    except Exception as e:
        print(f"❌ Backtest failed with error: {e}")
        import traceback
        traceback.print_exc()
        backtest_results = pd.DataFrame()
        performance_metrics = {}

else:
    print("❌ Cannot run backtest - strategy not initialized")
    backtest_results = pd.DataFrame()
    performance_metrics = {}

In [ ]:
# Visualize advanced strategy results
if not backtest_results.empty:
    fig, axes = plt.subplots(3, 2, figsize=(18, 16))
    fig.suptitle('Advanced Trading Strategy Results', fontsize=16, fontweight='bold')
    
    # Plot 1: Portfolio value over time
    axes[0, 0].plot(backtest_results.index, backtest_results['capital'], 
                   linewidth=2, color='blue', label='Strategy')
    
    # Add benchmark comparison
    if 'benchmark_returns' in locals():
        benchmark_capital = strategy.initial_capital * (1 + benchmark_returns).cumprod()
        axes[0, 0].plot(benchmark_capital.index, benchmark_capital.values, 
                       linewidth=2, color='red', alpha=0.7, label='Benchmark (Equal Weight)')
    
    axes[0, 0].axhline(y=strategy.initial_capital, color='gray', linestyle='--', alpha=0.5)
    axes[0, 0].set_title('Portfolio Value Over Time')
    axes[0, 0].set_ylabel('Capital ($)')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))
    
    # Plot 2: Daily returns distribution
    daily_returns = backtest_results['portfolio_return'].dropna()
    if len(daily_returns) > 0:
        axes[0, 1].hist(daily_returns, bins=50, alpha=0.7, color='blue', density=True)
        axes[0, 1].axvline(x=daily_returns.mean(), color='red', linestyle='--', 
                          label=f'Mean: {daily_returns.mean():.4f}')
        axes[0, 1].axvline(x=0, color='black', linestyle='-', alpha=0.5)
        axes[0, 1].set_title('Daily Returns Distribution')
        axes[0, 1].set_xlabel('Daily Return')
        axes[0, 1].set_ylabel('Density')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: Drawdown analysis
    capital_series = backtest_results['capital']
    running_max = capital_series.expanding().max()
    drawdowns = (capital_series - running_max) / running_max * 100
    
    axes[1, 0].fill_between(drawdowns.index, drawdowns.values, 0, 
                           alpha=0.5, color='red', label='Drawdown')
    axes[1, 0].axhline(y=-5, color='orange', linestyle='--', alpha=0.7, label='-5% Level')
    axes[1, 0].axhline(y=-10, color='red', linestyle='--', alpha=0.7, label='-10% Level')
    axes[1, 0].set_title('Portfolio Drawdowns')
    axes[1, 0].set_ylabel('Drawdown (%)')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Plot 4: Position allocation over time
    position_cols = [col for col in backtest_results.columns if col.endswith('_position')]
    if position_cols:
        for i, col in enumerate(position_cols[:4]):  # Show up to 4 assets
            asset_name = col.replace('_position', '')
            axes[1, 1].plot(backtest_results.index, backtest_results[col], 
                           label=asset_name, alpha=0.8, linewidth=1.5)
        
        axes[1, 1].axhline(y=0, color='black', linestyle='-', alpha=0.5)
        axes[1, 1].set_title('Position Allocation Over Time')
        axes[1, 1].set_ylabel('Position Size')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
    
    # Plot 5: Rolling Sharpe ratio
    if len(daily_returns) > 30:
        rolling_sharpe = (daily_returns.rolling(30).mean() / daily_returns.rolling(30).std()) * np.sqrt(252)
        axes[2, 0].plot(rolling_sharpe.index, rolling_sharpe.values, 
                       linewidth=2, color='purple', alpha=0.8)
        axes[2, 0].axhline(y=1, color='green', linestyle='--', alpha=0.7, label='Sharpe = 1.0')
        axes[2, 0].axhline(y=0, color='red', linestyle='--', alpha=0.7, label='Sharpe = 0.0')
        axes[2, 0].set_title('30-Day Rolling Sharpe Ratio')
        axes[2, 0].set_ylabel('Sharpe Ratio')
        axes[2, 0].legend()
        axes[2, 0].grid(True, alpha=0.3)
    
    # Plot 6: Trading activity
    axes[2, 1].bar(backtest_results.index, backtest_results['num_trades'], 
                  alpha=0.7, color='orange', width=1)
    axes[2, 1].set_title('Daily Trading Activity')
    axes[2, 1].set_ylabel('Number of Trades')
    axes[2, 1].set_xlabel('Date')
    axes[2, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Additional analysis: Regime analysis
    regime_cols = [col for col in backtest_results.columns if col.endswith('_regime')]
    
    if regime_cols:
        print(f"\n🔍 REGIME ANALYSIS DURING BACKTEST:")
        print(f"=" * 35)
        
        for col in regime_cols:
            asset_name = col.replace('_regime', '')
            regime_counts = backtest_results[col].value_counts().sort_index()
            
            print(f"\n{asset_name} Regime Distribution:")
            for regime, count in regime_counts.items():
                pct = count / len(backtest_results) * 100
                print(f"  State {int(regime)}: {count} days ({pct:.1f}%)")
    
    # Performance attribution by regime
    if regime_cols and 'portfolio_return' in backtest_results.columns:
        print(f"\n📊 PERFORMANCE BY MARKET REGIME:")
        print(f"=" * 35)
        
        # Use primary asset regime for analysis
        primary_regime_col = regime_cols[0]
        
        for regime in sorted(backtest_results[primary_regime_col].unique()):
            regime_mask = backtest_results[primary_regime_col] == regime
            regime_returns = backtest_results.loc[regime_mask, 'portfolio_return']
            
            if len(regime_returns) > 0:
                total_return = (1 + regime_returns).prod() - 1
                avg_daily_return = regime_returns.mean()
                volatility = regime_returns.std() * np.sqrt(252)
                
                print(f"\nRegime {int(regime)} ({len(regime_returns)} days):")
                print(f"  Total return: {total_return:.2%}")
                print(f"  Avg daily return: {avg_daily_return:.4f}")
                print(f"  Annualized volatility: {volatility:.2%}")
        
else:
    print("❌ No backtest results to visualize")

## 6. Summary and Best Practices

Let's summarize the advanced trading strategies and provide best practices for implementation.

In [ ]:
# Comprehensive summary and best practices
print(f"📚 ADVANCED TRADING STRATEGIES SUMMARY")
print(f"=" * 50)

print(f"\n✅ WHAT WE ACCOMPLISHED:")
print(f"1. Multi-Asset Regime Detection:")
print(f"   • Trained individual HMMs for {len(asset_hmms) if 'asset_hmms' in locals() else 0} assets")
print(f"   • Analyzed cross-asset regime correlations")
print(f"   • Identified regime transition patterns")

print(f"\n2. Advanced Position Sizing:")
print(f"   • Implemented regime-based position sizing")
print(f"   • Applied Kelly criterion optimization")
print(f"   • Integrated Value-at-Risk constraints")
print(f"   • Combined multiple position sizing approaches")

print(f"\n3. Portfolio Optimization:")
print(f"   • Regime-adjusted expected returns")
print(f"   • Dynamic covariance matrix estimation")
if optimization_results:
    print(f"   • Tested {len(optimization_results)} optimization methods")
    best_method = max(optimization_results.keys(), 
                     key=lambda k: optimization_results[k]['sharpe_ratio'])
    print(f"   • Best method: {best_method.replace('_', ' ').title()}")

print(f"\n4. Comprehensive Trading System:")
if 'performance_metrics' in locals() and performance_metrics:
    print(f"   • Executed {performance_metrics['total_trades']:.0f} trades")
    print(f"   • Achieved {performance_metrics['total_return']:.2%} total return")
    print(f"   • Sharpe ratio: {performance_metrics['sharpe_ratio']:.3f}")
    print(f"   • Maximum drawdown: {performance_metrics['max_drawdown']:.2%}")
else:
    print(f"   • Implemented complete backtesting framework")
    print(f"   • Integrated transaction costs and risk management")
    print(f"   • Real-time regime adaptation")

print(f"\n🎯 KEY INNOVATIONS:")
print(f"• Regime-Aware Position Sizing: Adapts leverage based on market conditions")
print(f"• Multi-Method Position Optimization: Combines regime, Kelly, and VaR approaches")
print(f"• Dynamic Portfolio Rebalancing: Adjusts allocations based on regime changes")
print(f"• Cross-Asset Regime Analysis: Identifies correlated market movements")
print(f"• Transaction Cost Integration: Realistic performance assessment")
print(f"• Risk-Adjusted Portfolio Construction: Regime-specific risk parameters")

print(f"\n⚙️ IMPLEMENTATION BEST PRACTICES:")
print(f"\n1. Data Quality and Preprocessing:")
print(f"   ✓ Use clean, adjusted price data")
print(f"   ✓ Handle missing data and outliers appropriately")
print(f"   ✓ Ensure sufficient historical data for HMM training")
print(f"   ✓ Validate data continuity and corporate actions")

print(f"\n2. Regime Detection:")
print(f"   ✓ Use 3-state models for interpretability")
print(f"   ✓ Require minimum 50-100 observations for training")
print(f"   ✓ Monitor regime detection confidence levels")
print(f"   ✓ Implement regime change detection alerts")
print(f"   ✓ Validate regime persistence and transitions")

print(f"\n3. Position Sizing:")
print(f"   ✓ Apply multiple position sizing methods")
print(f"   ✓ Use conservative approach (minimum of methods)")
print(f"   ✓ Set maximum position limits per asset")
print(f"   ✓ Implement regime-specific risk parameters")
print(f"   ✓ Monitor portfolio leverage continuously")

print(f"\n4. Risk Management:")
print(f"   ✓ Set portfolio-level risk limits")
print(f"   ✓ Implement stop-loss and take-profit mechanisms")
print(f"   ✓ Monitor correlation risk across positions")
print(f"   ✓ Stress test under different regime scenarios")
print(f"   ✓ Regular portfolio rebalancing based on regimes")

print(f"\n5. Transaction Costs:")
print(f"   ✓ Include realistic bid-ask spreads")
print(f"   ✓ Model market impact for large trades")
print(f"   ✓ Optimize rebalancing frequency")
print(f"   ✓ Implement minimum trade size thresholds")
print(f"   ✓ Consider timing of trade execution")

print(f"\n📊 PERFORMANCE EVALUATION FRAMEWORK:")
print(f"\nKey Metrics to Track:")
print(f"• Total and annualized returns")
print(f"• Risk-adjusted returns (Sharpe, Sortino ratios)")
print(f"• Maximum drawdown and recovery time")
print(f"• Win rate and profit factor")
print(f"• Portfolio turnover and transaction costs")
print(f"• Regime detection accuracy")
print(f"• Performance attribution by regime")

print(f"\n⚠️ COMMON PITFALLS TO AVOID:")
print(f"\n1. Overfitting:")
print(f"   ❌ Using too many states in HMMs")
print(f"   ❌ Over-optimizing on historical data")
print(f"   ❌ Ignoring out-of-sample validation")

print(f"\n2. Regime Detection Issues:")
print(f"   ❌ Assuming regime persistence is constant")
print(f"   ❌ Ignoring regime uncertainty")
print(f"   ❌ Not validating regime economic meaning")

print(f"\n3. Risk Management Failures:")
print(f"   ❌ Concentrating risk in correlated assets")
print(f"   ❌ Ignoring tail risks in regime transitions")
print(f"   ❌ Setting position sizes too aggressively")

print(f"\n4. Implementation Issues:")
print(f"   ❌ Ignoring transaction costs and market impact")
print(f"   ❌ Not accounting for realistic execution constraints")
print(f"   ❌ Over-trading due to regime noise")

print(f"\n🚀 PRODUCTION DEPLOYMENT CHECKLIST:")
print(f"□ Implement comprehensive data validation pipeline")
print(f"□ Set up real-time regime detection monitoring")
print(f"□ Configure position size limits and risk controls")
print(f"□ Establish performance monitoring and alerting")
print(f"□ Create regime change notification system")
print(f"□ Implement transaction cost tracking")
print(f"□ Set up automated rebalancing schedules")
print(f"□ Create emergency stop mechanisms")
print(f"□ Establish model validation procedures")
print(f"□ Document all parameters and assumptions")

print(f"\n📈 ADVANCED EXTENSIONS:")
print(f"1. Multi-Timeframe Analysis:")
print(f"   • Daily and intraday regime detection")
print(f"   • Long-term and short-term signal combination")
   
print(f"\n2. Alternative Data Integration:")
print(f"   • Economic indicators and sentiment data")
print(f"   • Options volatility surfaces")
print(f"   • News and social media sentiment")

print(f"\n3. Machine Learning Enhancements:")
print(f"   • Ensemble methods for regime detection")
print(f"   • Deep learning for feature extraction")
print(f"   • Reinforcement learning for position sizing")

print(f"\n4. Portfolio Construction:")
print(f"   • Multi-asset class allocation")
print(f"   • Factor-based portfolio construction")
print(f"   • ESG and sustainability constraints")

print(f"\n🎉 CONGRATULATIONS!")
print(f"You've mastered advanced trading strategies with HMM regime detection!")
print(f"\nNext steps:")
print(f"• Implement these strategies with your own data")
print(f"• Experiment with different parameter settings")
print(f"• Extend to additional asset classes and markets")
print(f"• Integrate with live trading platforms")
print(f"\nHappy trading! 🚀💰")